# Mask Detection with Neural Networks

Author: Umberto Michelucci - umberto.michelucci (at) gmail.com

Version 1.0

The code has been inspired, but simplified and modifed from https://www.pyimagesearch.com.

In [3]:
pip install tensorflow

In [4]:
pip install keras

Note: you may need to restart the kernel to use updated packages.


In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import tensorflow.keras as keras
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import cv2
import numpy as np
import os

ModuleNotFoundError: No module named 'tensorflow.keras'

In [5]:
INIT_LR = 1e-4 # learning rate
EPOCHS = 10 # 10 are more than enough to get an accuracy of over 99%
BS = 32 # Batchsize

## Data

We will use for this example data generated by Prajna Bhandary (you can get the data from her [GitHub repository](https://github.com/prajnasb/observations/tree/master/mask_classifier/Data_Generator)). This is a wonderful example of creativity in how to get enough data to train a deep learning model. The masks in the images are not real masks, but are artificially placed on the face of the person in the image, by identifying first the position of relevant landmarks of the face (nose, eyes, etc.) and then placing a "fake" masks on top of the face. 

In [6]:
print("[INFO] loading images...")
imagePaths = list(paths.list_images("data"))
data = []
labels = []
# loop over the image paths
for imagePath in imagePaths:
    # extract the class label from the filename
    label = imagePath.split(os.path.sep)[-2]
    # Since we are going to use MobileNetV2 we need to resize the images
    # to the expected size by the pre-trained network.
    image = load_img(imagePath, target_size=(224, 224))
    image = img_to_array(image)
    image = preprocess_input(image)
    data.append(image)
    labels.append(label)
# convert the data and labels to NumPy arrays
data = np.array(data, dtype="float32")
labels = np.array(labels)

[INFO] loading images...


NameError: name 'paths' is not defined

In [ ]:
data.shape

Now we need to split the dataset to check overfitting and then we can create a `ImageDataGenerator` 
to do data augmentation.

In [ ]:
# perform one-hot encoding on the labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)
# Split of data into train and test
(trainX, testX, trainY, testY) = train_test_split(data, labels,
    test_size=0.20, stratify=labels, random_state=42)
# construct the training image generator for data augmentation
aug = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest")

In [ ]:
# We first load the MobilNetV2 by removing the last layers since
# we want to do transfer learning.
baseModel = MobileNetV2(weights="imagenet", include_top=False,
    input_tensor=Input(shape=(224, 224, 3)))
# The following are the layers we will train
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation="softmax")(headModel)
model = Model(inputs=baseModel.input, outputs=headModel)
# We don't want to train any layer in the MobileNetV2 network
for layer in baseModel.layers:
    layer.trainable = False


In [ ]:
model.summary()

In [ ]:
# compile our model
opt = Adam(learning_rate=INIT_LR, weight_decay=INIT_LR / EPOCHS)
model.compile(loss="binary_crossentropy", optimizer=opt,
    metrics=["accuracy"])
H = model.fit(
    aug.flow(trainX, trainY, batch_size=BS),
    steps_per_epoch=len(trainX) // BS,
    validation_data=(testX, testY),
    validation_steps=len(testX) // BS,
    epochs=EPOCHS)

A quick reminder

- TP: True Positive
- FP: False Positive
- TN: True Negative
- FN: False Negative


$$
\textrm{PRECISION} = \frac{TP}{TP+FP}
$$


$$
\textrm{RECALL} = \frac{TP}{P} = \frac{TP}{TP+FN}
$$

In [ ]:
print("[INFO] evaluating network...")
predIdxs = model.predict(testX, batch_size=BS)
predIdxs = np.argmax(predIdxs, axis=1)
print(classification_report(testY.argmax(axis=1), predIdxs,
    target_names=lb.classes_))
model.save("model_mask_detection", save_format="h5")

In [ ]:
model = keras.models.load_model('model_mask_detection')

In [ ]:
N = EPOCHS
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="upper right")
plt.savefig("losses.png")

# Test of model

In [ ]:
face = cv2.imread('test.jpg')
face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
face = cv2.resize(face, (224, 224))


face2 = img_to_array(face)
face2 = preprocess_input(face2)
face2 = np.expand_dims(face2, axis=0)
pred = np.argmin(model.predict(face2))
print(pred)

In [ ]:
plt.imshow(face)
plt.grid(False)
title = 'With Mask' if (pred == 1) else 'Without Mask'
plt.title(title)
plt.show()

In [ ]:
face = cv2.imread('test_no.jpg')
face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
face = cv2.resize(face, (224, 224))


face2 = img_to_array(face)
face2 = preprocess_input(face2)
face2 = np.expand_dims(face2, axis=0)
pred = np.argmin(model.predict(face2))
print(pred)

In [ ]:
plt.imshow(face)
plt.grid(False)
title = 'With Mask' if (pred == 1) else 'Without Mask'
plt.title(title)
plt.show()

## Example with mask

In [ ]:
face = cv2.imread('data/with_mask/0-with-mask.jpg')
face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
face = cv2.resize(face, (224, 224))


face2 = img_to_array(face)
face2 = preprocess_input(face2)
face2 = np.expand_dims(face2, axis=0)
pred = np.argmin(model.predict(face2))
print(pred)

In [ ]:
plt.imshow(face)
plt.grid(False)
title = 'With Mask' if (pred == 1) else 'Without Mask'
plt.title(title)
plt.show()

## Example without mask

In [ ]:
face = cv2.imread('data/without_mask/0.jpg')
face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
face = cv2.resize(face, (224, 224))


face2 = img_to_array(face)
face2 = preprocess_input(face2)
face2 = np.expand_dims(face2, axis=0)
pred = np.argmin(model.predict(face2))
print(pred)

In [ ]:
plt.imshow(face)
plt.grid(False)
title = 'With Mask' if (pred == 1) else 'Without Mask'
plt.title(title)
plt.show()

# Improvements and Exercises

1. The training data is limited. More data with different pose and skin colors would increase the accuracy of the model. Can you create a larger dataset with the same approach?
2. The model works best with white masks (the data is using only white masks). Generating images with masks of different colors would increase further the accuracy of the model. Try to trick the model by using different colors of masks. Does the model works equally good? Try to create a larger dataset by using masks of different colors and then train again the model. Can you get better results with masks that are not white?
3. How could you get better? Maybe you could first localize the face and then apply the model you just trained only to the face (and not the entire image). But what happens if the mask covers so much face that you cannot localize it in the image?
4. Can you make the model better to make it work with multiple persons in the image? You have to first implement point 3. and then do a loop over all the faces found. 